In [ ]:
import celloracle as co

In [ ]:
co.network_analysis.set_R_path("/opt/R/4.0.4/bin/R")

In [ ]:
co.test_R_libraries_installation()

In [ ]:
co.check_python_requirements()

# Prepare Base GRN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

from celloracle import motif_analysis as ma

In [ ]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [ ]:
# Load scATAC-seq peak list.
peaks = pd.read_csv("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle/all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

In [ ]:
# Load cicero coaccess score.
cicero_connections = pd.read_csv("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle/cicero_connections.csv", index_col=0)
cicero_connections.head()

# Make TSS annotation

In [ ]:
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="hg38")

In [ ]:
# Check results
tss_annotated.tail()

# Integrate TSS info and cicero connections

In [ ]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated,
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

# Filter peaks

In [ ]:
#Remove peaks that have weak coaccess score
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [ ]:
print(peak.shape)
peak.head()

In [ ]:
#Save the promoter/enhancer peak.
peak.to_csv("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/processed_peak_file.csv")

# Motif Scan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

In [ ]:
# Reference genome data preparation
ref_genome = "hg38"
import genomepy
genomepy.install_genome("hg38", "UCSC")

In [ ]:
# Load annotated peak data.
peaks = pd.read_csv("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/processed_peak_file.csv", index_col=0)
peaks.head()

In [ ]:
# Define function for quality check
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'

    Returns:
        tuple: chromosome name, start position, end position
    """

    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_foamat(peaks_df, ref_genome):
    """
    Check peak fomat.
     (1) Check chromosome name.
     (2) Check peak size (length) and remove sort DNAs (<5bp)

    """

    df = peaks_df.copy()

    n_peaks_before = df.shape[0]

    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed))
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(np.int)
    df_decomposed["end"] = df_decomposed["end"].astype(np.int)

    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())


    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])


    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]

    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])

    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]

    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)

    return df

In [ ]:
peaks = check_peak_foamat(peaks, ref_genome)

In [ ]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome)

In [ ]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/celloracle.tfinfo")

In [ ]:
# Check motif scan results
tfi.scanned_df.head()

In [ ]:
#Filtering motifs

# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Do post filtering process. Convert results into several file format.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)


In [ ]:
#Get Final results
df = tfi.to_dataframe()
df.head()

In [ ]:
# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/base_GRN_dataframe.parquet")


# GRN Model Construction and Network Analysis

In [ ]:
# 0. Import

import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

In [ ]:
import celloracle as co
co.__version__

In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [ ]:
co.test_R_libraries_installation()

In [ ]:
save_folder = "/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/Figures"
os.makedirs(save_folder, exist_ok=True)

In [ ]:
#/media/Home_Raid1_Voyager/b1lake/anaconda3/envs/celloracle_env/bin/seuratToAnndata "/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle/aTAL_Seurat.Rds" "/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle/aTALAnnData"

In [ ]:
# Load data
adata = sc.read_h5ad("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle/aTALAnnData.h5ad")

In [ ]:
adata

In [ ]:
print(f"Cell number is :{adata.shape[0]}")
print(f"Gene number is :{adata.shape[1]}")

In [ ]:
base_GRN = pd.read_parquet("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/base_GRN_dataframe.parquet")
base_GRN.head()

In [ ]:
#Initiate Oracle object
oracle = co.Oracle()

In [ ]:
# Show data name in anndata
print("metadata columns :", list(adata.obs.columns))
print("dimensional reduction: ", list(adata.obsm.keys()))

In [ ]:
adata.obs.columns

In [ ]:
# Check current cluster name
cluster_list = adata.obs.modules.unique()
cluster_list

In [ ]:
adata.obsm['umap']=adata.obsm['X_umap.traj']
sc.pl.umap(adata, color="modules")

In [ ]:
# In this notebook, we use raw mRNA count as an input of Oracle object.
adata.X = adata.layers["raw_count"].copy()

# Instantiate Oracle object.
oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name="modules",
                                   embedding_name="X_umap.traj")

In [ ]:
# Load TF info dataframe
oracle.import_TF_data(TF_info_matrix=base_GRN)


In [ ]:
#Knn imputation
# Perform PCA
oracle.perform_PCA()

# Select important PCs
plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
plt.axvline(n_comps, c="k")
print(n_comps)
n_comps = min(n_comps, 50)

In [ ]:
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")

In [ ]:
k = int(0.025*n_cell)
print(f"Auto-selected k is :{k}")

In [ ]:
oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                      b_maxl=k*4, n_jobs=4)

In [ ]:
# Save oracle object.
oracle.to_hdf5("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/aTAL_Trajectory.celloracle.oracle")

In [ ]:
# Load file.
oracle = co.load_hdf5("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/aTAL_Trajectory.celloracle.oracle")

# GRN calculation

In [ ]:
%%time
# Calculate GRN for each population in "louvain_annot" clustering unit.
# This step may take long time.(~30 minutes)
links = oracle.get_links(cluster_name_for_GRN_unit="modules", alpha=10,
                         verbose_level=10, test_mode=False)

In [ ]:
# Show the contents of pallete
links.palette

In [ ]:
# Change the order of pallete
order = ['black', 'pink', 'brown', 'yellow', 'blue']
links.palette = links.palette.loc[order]
links.palette

In [ ]:
links.palette.loc['black'] = 'black'
links.palette.loc['pink'] = 'pink'
links.palette.loc['brown'] = 'brown'
links.palette.loc['yellow'] = 'yellow'
links.palette.loc['blue'] = 'blue'
links.palette

In [ ]:
#Network preprocessing
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)

In [ ]:
plt.rcParams["figure.figsize"] = [9, 4.5]

In [ ]:
links.plot_degree_distributions(plot_model=True,
                                               #save=f"{save_folder}/degree_distribution/",
                                               )

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4.5]

In [ ]:
# Calculate network scores. It takes several minutes.
links.get_score()

In [ ]:
links.merged_score.head()

In [ ]:
# Save as csv
links.merged_score.to_csv(f"{save_folder}/Merged_Links_Scores_for_Modules.csv")

In [ ]:
# Save Links object.
links.to_hdf5(file_path="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/aTAL_Trajectory_links.celloracle.links")

In [ ]:
# You can load files with the following command.
links = co.load_hdf5(file_path="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/Celloracle2/aTAL_Trajectory_links.celloracle.links")

# Network analysis; Network score for each gene

In [ ]:
#visualize genes with high network centrality
# Check cluster name
links.cluster

In [ ]:
links.thread_number = [1]

In [ ]:
# Visualize top n-th genes that have high scores.
links.plot_scores_as_rank(cluster="black", n_gene=30, save=f"{save_folder}/ranked_score")

In [ ]:
# Visualize top n-th genes that have high scores.
links.plot_scores_as_rank(cluster="yellow", n_gene=30, save=f"{save_folder}/ranked_score")

In [ ]:
# Visualize top n-th genes that have high scores.
links.plot_scores_as_rank(cluster="blue", n_gene=30, save=f"{save_folder}/ranked_score")

In [ ]:
# Visualize top n-th genes that have high scores.
links.plot_scores_as_rank(cluster="pink", n_gene=30, save=f"{save_folder}/ranked_score")

In [ ]:
# Visualize top n-th genes that have high scores.
links.plot_scores_as_rank(cluster="brown", n_gene=30, save=f"{save_folder}/ranked_score")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="degree_centrality_all",
                               cluster1="pink", cluster2="blue",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="eigenvector_centrality",
                               cluster1="pink", cluster2="blue",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="betweenness_centrality",
                               cluster1="pink", cluster2="blue",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="degree_centrality_all",
                               cluster1="pink", cluster2="yellow",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="eigenvector_centrality",
                               cluster1="pink", cluster2="yellow",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="betweenness_centrality",
                               cluster1="pink", cluster2="yellow",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="degree_centrality_all",
                               cluster1="pink", cluster2="brown",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="eigenvector_centrality",
                               cluster1="pink", cluster2="brown",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
links.plot_score_comparison_2D(value="betweenness_centrality",
                               cluster1="pink", cluster2="brown",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="EGF", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="DCDC2", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="PROM1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="ERBB4", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="PPARGC1A", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="STAT3", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="NR3C1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="NR3C2", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="NR2F2", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="ESRRG", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="SPP1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="WNK1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="ELF1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="TFAP2B", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="ZEB1", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="TCF7L2", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="ESRRB", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="NPAS3", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize network score dynamics
links.plot_score_per_cluster(goi="FOXK2", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
cluster_name = "blue"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()
# Save as csv
filtered_links_df.to_csv(f"{save_folder}/Filtered_Links_for_{cluster_name}_Module.csv")

In [ ]:
cluster_name = "yellow"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()
# Save as csv
filtered_links_df.to_csv(f"{save_folder}/Filtered_Links_for_{cluster_name}_Module.csv")

In [ ]:
cluster_name = "brown"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()
# Save as csv
filtered_links_df.to_csv(f"{save_folder}/Filtered_Links_for_{cluster_name}_Module.csv")

In [ ]:
cluster_name = "pink"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()
# Save as csv
filtered_links_df.to_csv(f"{save_folder}/Filtered_Links_for_{cluster_name}_Module.csv")

In [ ]:
cluster_name = "black"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()
# Save as csv
filtered_links_df.to_csv(f"{save_folder}/Filtered_Links_for_{cluster_name}_Module.csv")